In [ ]:
!pixi install 
# set up environment 
# to use that environment in a shell, use !pixi shell
# or, use !pixi run [my_program].py

# How to download archived VIIRS Active Fire data from UMD FTP server

Reference: Collection 2 Visible Infrared Imaging Radiometer Suite (VIIRS)
375-m Active Fire Product User’s Guide
Version 1.0 (May 2024) https://viirsland.gsfc.nasa.gov/PDF/VIIRS_activefire_User_Guide.pdf

From Julia: "It would be interesting to try to use the fsspec sftp file system to manage this download process."

VNP14IMG == SNPP

VJ14IMG == NOAA20

VJ24IMG == NOAA21 

VNP14IMG ML = monthly location summary files in CSV format. Naming: Vxx14IMGML.YYMM.CC.VV.csv, where xx = platform (NP, J1, J2 for SNPP, NOAA20, NOAA21), YYMM = year and month of data acquisition, CC = collection number (currently 02) and VV = product versio number (currently 02)

Date range: 20 Jan 2012 to 31 Dec 2024 if available


The VNP14IMGML product can be downloaded by connecting to an SFTP server at the University
of Maryland. The login information is as follows:

- server: fuoco.geog.umd.edu
- login: fire
- password: burnt
- directory: data/VIIRS/C2/VNP14IMGML

New VNP14IMGML files are created regularly and posted to the SFTP server above 1–2
months after all Level-2 granules for a given calendar month are generated by NASA.


In [ ]:
import fsspec
import os

# connect to UMD SFTP server 
fs = fsspec.filesystem('sftp', host="fuoco.geog.umd.edu", username="fire", password="burnt")

using fsspec, we can navigate the filesystem of the remote server in typical unix fashion

`fs.ls('/')`

['/data', '/fire']

In [ ]:
# we want the collection 2 data from SNPP
snpp = fs.ls('/data/VIIRS/C2/VNP14IMGML/')
print(f"{len(snpp)} monthly files availabe.\n{snpp[0]} to {snpp[-1]}")

In [ ]:
os.mkdir("./target")

# here, we are copying the whole directory of SNPP monthly archive files 
# to our local machine- this may take a while 
fs.get("/data/VIIRS/C2/VNP14IMGML/", "./target/", recursive=True)
# took me 13 minutes, so not too bad 

## Scratch work: 

In [ ]:
# dask dataframes can read from globs of compressed files
import dask.dataframe as dd

df = dd.read_csv("./target/*.csv.gz", compression="gzip")
df
